In [68]:
import seaborn as sns
from sklearn import datasets # sample data
import numpy as np
import scikit_posthocs as sp
from scipy import stats as st
import statsmodels.formula.api as smf
import statsmodels.api as sm
import statsmodels.stats.anova as anova
from statsmodels.stats.multicomp import pairwise_tukeyhsd

iris = sns.load_dataset('iris')
iris['index'] = iris.index
print(iris)

     sepal_length  sepal_width  petal_length  petal_width    species  index
0             5.1          3.5           1.4          0.2     setosa      0
1             4.9          3.0           1.4          0.2     setosa      1
2             4.7          3.2           1.3          0.2     setosa      2
3             4.6          3.1           1.5          0.2     setosa      3
4             5.0          3.6           1.4          0.2     setosa      4
..            ...          ...           ...          ...        ...    ...
145           6.7          3.0           5.2          2.3  virginica    145
146           6.3          2.5           5.0          1.9  virginica    146
147           6.5          3.0           5.2          2.0  virginica    147
148           6.2          3.4           5.4          2.3  virginica    148
149           5.9          3.0           5.1          1.8  virginica    149

[150 rows x 6 columns]


In [69]:
species = ['setosa', 'versicolor', 'virginica']
factors = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
level = 0.05 #有意水準

# 正規性の検定
# shapiro-wilk testを採用，データ数が数千までに上場合は，Kolmogorov-Smirnov testの方が好ましい
print('------------------------------ shapiro-wilk test ------------------------------')
#以下は，各種の各パラメータに対して検定
print('--------------- parameter per species ---------------' )
for content in species:
    for factor in factors:
        statistic, pvalue = st.shapiro(iris.query('species == @content')[factor])
        print('{0}-{1} \n statistic: {2}, \n p-value: {3}{4}'.format(content,factor,statistic,pvalue,'*' if pvalue < level else ''))
    print('--------------------')
# result
# setosa: petal_widthに有意差あり→ノンパラメトリック 
# versicolor: petal_width に有意差あり→ノンパラメトリック 
# virginica: 全て有意差なし

#以下は，種は分けずに各パラメータに対して検定
print('---------------  per parameter ---------------')
for factor in factors:
      statistic, pvalue = st.shapiro(iris[factor])
      print('{0} \n statistic: {1}, \n p-value: {2}{3}'.format(factor,statistic,pvalue,'*' if pvalue < level else ''))
      print('--------------------')
# result
# sepal_length: 有意差あり→ノンパラメトリック 
# sepal_width : 有意差なし→パラメトリック
# petal_length : 有意差あり→ノンパラメトリック 
# petal_width : 有意差あり→ノンパラメトリック 

------------------------------ shapiro-wilk test ------------------------------
--------------- parameter per species ---------------
setosa-sepal_length 
 statistic: 0.9776989221572876, 
 p-value: 0.4595281183719635
setosa-sepal_width 
 statistic: 0.97171950340271, 
 p-value: 0.2715264856815338
setosa-petal_length 
 statistic: 0.9549766182899475, 
 p-value: 0.05481043830513954
setosa-petal_width 
 statistic: 0.7997642159461975, 
 p-value: 8.65842082475865e-07*
--------------------
versicolor-sepal_length 
 statistic: 0.9778355956077576, 
 p-value: 0.46473264694213867
versicolor-sepal_width 
 statistic: 0.9741330742835999, 
 p-value: 0.33798879384994507
versicolor-petal_length 
 statistic: 0.9660047888755798, 
 p-value: 0.1584833413362503
versicolor-petal_width 
 statistic: 0.947626531124115, 
 p-value: 0.027278218418359756*
--------------------
virginica-sepal_length 
 statistic: 0.9711798429489136, 
 p-value: 0.25832483172416687
virginica-sepal_width 
 statistic: 0.9673910140991211, 

In [84]:
# 対応ありver

#他群検定 (3種全部で行う)

# sepal_length - フリードマン検定
print('------------------------------ friedman test - sepal_length ------------------------------')
statistic, pvalue = st.friedmanchisquare(*(x[1] for x in iris.groupby('species')['sepal_length']))
print('sepal_length friedman test \n statistic: {0}, p-value: {1}{2}'.format(statistic,pvalue, '*' if pvalue<level else ''))
# result: 有意差あり→多重検定を行う
print('sepal_length Steel-Dwass test: ')
print(sp.posthoc_dscf(iris, val_col='sepal_length', group_col='species'))



# sepal_width - 反復測定ANOVA
# print('------------------------------ Repeated-measures ANOVA - sepal_width ------------------------------')
# print('sepal_width One-Factor repeated measures ANOVA: ')
# statistic, pvalue = st.f_oneway(*(x[1] for x in iris['sepal_width']))



# petal_length - フリードマン検定
print('------------------------------ friedman test - petal_length  ------------------------------')
statistic, pvalue = st.friedmanchisquare(*(x[1] for x in iris.groupby('species')['petal_length']))
print('petal_length friedman test \n statistic: {0}, p-value: {1}{2}'.format(statistic,pvalue, '*' if pvalue<level else ''))
# result: 有意差あり→多重検定 Steel Dwassを行う
print('petal_length Steel-Dwass test: ')
print(sp.posthoc_dscf(iris, val_col='petal_length', group_col='species'))



# petal_width - フリードマン検定
print('------------------------------ friedman test - petal_length  ------------------------------')
statistic, pvalue = st.friedmanchisquare(*(x[1] for x in iris.groupby('species')['petal_width']))
print('petal_width friedman test \n statistic: {0}, p-value: {1}{2}'.format(statistic,pvalue, '*' if pvalue<level else ''))
# result: 有意差あり→多重検定 Steel Dwassを行う
print('petal_width Steel-Dwass test: ')
print(sp.posthoc_dscf(iris, val_col='petal_width', group_col='species'))

------------------------------ friedman test - sepal_length ------------------------------
sepal_length friedman test 
 statistic: 73.78571428571435, p-value: 9.498077769131953e-17*
sepal_length Steel-Dwass test: 
            setosa  versicolor  virginica
setosa       1.000       0.001      0.001
versicolor   0.001       1.000      0.001
virginica    0.001       0.001      1.000
------------------------------ friedman test - petal_length  ------------------------------
petal_length friedman test 
 statistic: 95.31313131313132, p-value: 2.0091691859708084e-21*
petal_length Steel-Dwass test: 
            setosa  versicolor  virginica
setosa       1.000       0.001      0.001
versicolor   0.001       1.000      0.001
virginica    0.001       0.001      1.000
------------------------------ friedman test - petal_length  ------------------------------
petal_width friedman test 
 statistic: 96.15999999999997, p-value: 1.315592261061294e-21*
petal_width Steel-Dwass test: 
            setosa  v